# Running a Simple Simulation and Recording Spikes

**Objective:** This notebook demonstrates how to take an organoid created with `pybrainorg` (as in Notebook 01), set up a simulation environment, run a basic simulation, and record/visualize neuronal spikes.

**Key `pybrainorg` components used:**
- `pybrainorg.organoid.Organoid`
- `pybrainorg.core.neuron_models`
- `pybrainorg.organoid.spatial`
- `pybrainorg.simulation.Simulator`: The class to orchestrate simulations.
- `pybrainorg.electrophysiology.brian_monitors`: Helper functions to set up monitors.
- `pybrainorg.visualization.spike_plotter`: For plotting raster plots.

## 1. Imports and Setup

Import necessary modules from `brian2`, `pybrainorg`, and `matplotlib`.

In [ ]:
import brian2 as b2
import numpy as np
import matplotlib.pyplot as plt

from pybrainorg.organoid.organoid import Organoid
from pybrainorg.core import neuron_models
from pybrainorg.organoid import spatial
from pybrainorg.simulation.simulator import Simulator
from pybrainorg.electrophysiology import brian_monitors # For monitor setup functions
from pybrainorg.visualization import spike_plotter # For plotting

b2.prefs.codegen.target = 'numpy'
b2.seed(123) # Use a different seed or same for direct comparison
np.random.seed(123)

## 2. Create an Organoid

We'll recreate a simple organoid similar to Notebook 01. This time, we'll prepare it for some activity by setting an initial tonic current or ensuring the initial Vm is close to threshold for some neurons.

In [ ]:
simple_organoid = Organoid(name="SimOrganoid")

num_neurons = 50
lif_params_active = {
    'tau_m': 15*b2.ms,
    'v_rest': -70*b2.mV,
    'v_reset': -70*b2.mV,
    'v_thresh': -55*b2.mV,
    'R_m': 150*b2.Mohm,
    'I_tonic': 0.11*b2.nA, # A small tonic current to make them somewhat active
    'refractory_period': 3*b2.ms
}

neuron_positions = spatial.random_positions_in_cube(
    N=num_neurons, 
    side_length=100*b2.um
)

ng = simple_organoid.add_neurons(
    name="active_lif_neurons",
    num_neurons=num_neurons,
    model_name="LIFNeuron",
    model_params=lif_params_active,
    positions=neuron_positions,
    # Set slightly varied initial Vm to desynchronize initial spikes if I_tonic is high
    initial_values={'v': '-70*mV + rand()*5*mV'}
)

print(f"Created organoid: {simple_organoid} with neuron group: {ng.name}")

## 3. Instantiate the Simulator

Create a `Simulator` instance, passing our organoid to it. We can also specify a `brian2_dt` for the simulation if needed (otherwise, Brian2's default is used).

In [ ]:
simulation_dt = 0.1*b2.ms
my_simulator = Simulator(organoid=simple_organoid, brian2_dt=simulation_dt)
print(my_simulator)

## 4. Add Monitors for Recording

We need to tell the simulator what data to record. We'll add a `SpikeMonitor` to record all spikes from our neuron population and a `StateMonitor` to record the membrane potential (Vm) of a few selected neurons.

In [ ]:
# Add a SpikeMonitor
# The monitor_name is how we will retrieve its data later.
# target_group_name is the name we gave to our NeuronGroup in the Organoid.
spike_mon = my_simulator.add_recording(
    monitor_name="all_spikes",
    monitor_type="spike",
    target_group_name="active_lif_neurons",
    record=True # Record from all neurons in the group
)
print(f"Added SpikeMonitor: {spike_mon.name}")

# Add a StateMonitor for Vm of first 3 neurons
state_mon_vm = my_simulator.add_recording(
    monitor_name="vm_traces_subset",
    monitor_type="state",
    target_group_name="active_lif_neurons",
    variables='v', # Variable to record (must exist in neuron model)
    record=[0, 1, 2], # Record Vm for neurons with index 0, 1, and 2
    dt=0.5*b2.ms # Optional: record Vm at a specific dt (can be > simulation_dt)
)
print(f"Added StateMonitor for Vm: {state_mon_vm.name}")

print(f"Simulator monitors: {my_simulator.monitors.keys()}")

## 5. Run the Simulation

Now we can run the simulation for a specified duration. The `Simulator` will build the Brian2 `Network` internally if it hasn't been built yet.

In [ ]:
simulation_duration = 200*b2.ms

print(f"Starting simulation for {simulation_duration}...")
my_simulator.run(simulation_duration, report='text', report_period=50*b2.ms)
print("Simulation finished.")

## 6. Retrieve and Visualize Recorded Data

After the simulation, we can get the data from our monitors using the names we assigned.

### 6.1. Spike Data (Raster Plot)

In [ ]:
recorded_spikes = my_simulator.get_data("all_spikes")

print(f"Number of spikes recorded: {len(recorded_spikes.i)}")
if len(recorded_spikes.i) > 0:
    print(f"Example spike indices: {recorded_spikes.i[:5]}")
    print(f"Example spike times: {recorded_spikes.t[:5]}")

    fig_raster, ax_raster = plt.subplots(figsize=(12, 5))
    spike_plotter.plot_raster(
        spike_indices=recorded_spikes.i,
        spike_times=recorded_spikes.t,
        duration=simulation_duration,
        ax=ax_raster,
        title="Spike Raster Plot of LIF Neurons"
    )
    plt.tight_layout()
    plt.show()
else:
    print("No spikes were recorded.")

### 6.2. Membrane Potential (Vm) Traces

In [ ]:
recorded_vm = my_simulator.get_data("vm_traces_subset")

print(f"Shape of recorded Vm data (neurons, timepoints): {recorded_vm.v.shape}")
print(f"Time points for Vm: {recorded_vm.t[:5]} ...")

fig_vm, ax_vm = plt.subplots(figsize=(12, 5))
spike_plotter.plot_vm_traces(
    state_monitor=recorded_vm,
    # neuron_indices=None, # plot_vm_traces will plot all recorded if few, here we recorded 3
    ax=ax_vm,
    title="Membrane Potential (Vm) of Selected Neurons"
)
plt.tight_layout()
plt.show()

## 7. Summary

In this notebook, we have:
1. Created a `pybrainorg.Organoid` with a population of LIF neurons configured to be active.
2. Instantiated a `pybrainorg.simulation.Simulator`.
3. Added a `SpikeMonitor` and a `StateMonitor` to record activity.
4. Executed a simulation for a defined duration.
5. Retrieved the recorded spike and Vm data.
6. Visualized the spikes using a raster plot and Vm traces using `pybrainorg.visualization.spike_plotter`.

This forms the basic workflow for setting up and running simulations with `pybrainorg`.